### Download

In [ ]:
import paramiko

USERNAME = 'your-username-on-NAS'
PASSWORD = 'your-password-on-NAS'

with paramiko.Transport(('10.0.0.202', 5812)) as t:
    t.connect(username=USERNAME, password=PASSWORD)
    sftp = paramiko.SFTPClient.from_transport(t)
    sftp.get('/homes/jacob.lin/jupyterlab-crypten-0.1-jacob/model_alice.pth', './model_alice.pth')
    sftp.get('/homes/jacob.lin/jupyterlab-crypten-0.1-jacob/split.json', './split.json')
    sftp.get('/homes/datasets/flower/flower_photos.tgz', './flower_photos.tgz')

!tar xfz flower_photos.tgz

### Setup

In [ ]:
!ip addr

In [ ]:
import os

os.environ["RENDEZVOUS"] = "tcp://{IP-address-of-Alice}:2345"
os.environ["WORLD_SIZE"] = "2"
os.environ["RANK"] = "{0-or-1}"

In [ ]:
import crypten
import torch
import time

# Establish the communication channel, set seeds for random number generators, etc.
crypten.init()

### Define model structure

In [ ]:
# Define ResNet
import torch.nn as nn
import torch.nn.functional as F

def _conv1x1(inplanes, out_planes, stride=1):
    return nn.Conv2d(inplanes, out_planes, kernel_size=1, stride=stride, bias=False)

def _conv3x3(inplanes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(inplanes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = _conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = _conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

        self.shortcut = nn.Sequential()
        if not stride == 1 or not inplanes == self.expansion*planes:
            self.shortcut = nn.Sequential(
                    nn.Conv2d(inplanes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(self.expansion*planes)
                    )
    
    def forward(self, x):
        if self.downsample is None:
            identity = x
        else:
            identity = self.downsample(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        return out

class ResNetArch(nn.Module):
    def __init__(self, block, n_blocks, num_classes):
        super(ResNetArch, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, n_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, n_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, n_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, n_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, n_block, stride):
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                _conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion)
            )
        else:
            downsample = None
    
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, n_block):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

crypten.common.serial.register_safe_class(ResNetArch)

### Metric

In [ ]:
def compute_accuracy(output, labels):
    pred = output.argmax(1)
    correct = pred.eq(labels)
    correct_count = correct.sum(0, keepdim=True).float()
    accuracy = correct_count.mul_(100.0 / output.size(0))
    return accuracy

### Dataloader

In [ ]:
import os
import yaml
import json
import pickle as pkl
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as torch_transforms

SPLIT_FILE = './split.json'

def load_file(path):
    loader = {"yaml": yaml_load,
              "pkl": pkl_load,
              "json": json_load}
    ext = os.path.basename(path).split(".")[-1]
    if ext not in loader:
        raise RuntimeError("File extension is not supported by loader")
    return loader[ext](path)

def json_load(path):
    return json.load(open(path))

def yaml_load(path):
    return yaml.full_load(open(path))

def pkl_load(path):
    return pkl.load(open(path, "rb"))

class FlowerData(Dataset):
    DEFAULTS = {}
    def __init__(self, mode):
        self.__dict__.update(self.DEFAULTS)
        try:
            split_dict = load_file(SPLIT_FILE)
            self.examples = split_dict[mode]
        except:
            self.examples = {}

        transforms = {"train": torch_transforms.Compose([
                                   torch_transforms.Resize(256),
                                   torch_transforms.RandomCrop(224),
                                   torch_transforms.RandomHorizontalFlip(),
                                   torch_transforms.ToTensor(),
                                   torch_transforms.Normalize((0.6959, 0.6537, 0.6371), (0.3113, 0.3192, 0.3214))
                               ]),
                      "test": torch_transforms.Compose([
                                  torch_transforms.Resize(256),
                                  torch_transforms.CenterCrop(224),
                                  torch_transforms.ToTensor(),
                                  torch_transforms.Normalize((0.6959, 0.6537, 0.6371), (0.3113, 0.3192, 0.3214))
                              ])
        }
        self.transforms = transforms[mode]
        
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, index):
        img_path, label = self.examples[index]
        img = Image.open(img_path).convert("RGB")
        img = self.transforms(img)
        return img, label

### Encrypted Dataloader

In [ ]:
class FlowerDataEnc(FlowerData):
    DEFAULTS = {}
    def __init__(self, mode, src):
        super(FlowerDataEnc, self).__init__(mode)
        self.src = src
        
    def __getitem__(self, index):
        # Only Bob has access to the data
        try:
            img_path, label = self.examples[index]
            img = Image.open(img_path).convert("RGB")
        except:
            img = Image.new("RGB", (256, 256), (255, 255, 255))
        img = self.transforms(img)
        img_enc = crypten.cryptensor(img, src=self.src)
        label_enc = crypten.cryptensor(label, src=self.src)
        return img_enc, label_enc

def crypten_collate(batch):
    elem = batch[0]
    elem_type = type(elem)

    if isinstance(elem, crypten.CrypTensor):
        return crypten.stack(list(batch), dim=0)

    elif isinstance(elem, typing.Sequence):
        size = len(elem)
        assert all(len(b) == size for b in batch), "each element in list of batch should be of equal size"
        transposed = zip(*batch)
        return [crypten_collate(samples) for samples in transposed]

    elif isinstance(elem, typing.Mapping):
        return {key: crypten_collate([b[key] for b in batch]) for key in elem}

    elif isinstance(elem, tuple) and hasattr(elem, '_fields'):  # namedtuple
        return elem_type(*(crypten_collate(samples) for samples in zip(*batch)))

    return "crypten_collate: batch must contain CrypTensor, dicts or lists; found {}".format(elem_type)

### ciphertext inference

In the multiprocessing environment, `ALICE` corresponds to rank `0`, and `BOB` corresponds to rank `1`.

In [ ]:
import typing
import crypten.mpc as mpc
import crypten.communicator as comm

In [ ]:
dummy_model = ResNetArch(block=BasicBlock, n_blocks=[2, 2, 2, 2], num_classes=5)

# Only Alice need to load the trained model
try:
    model_ckpt = torch.load('./model_alice.pth', map_location='cpu')
    dummy_model.load_state_dict(model_ckpt["model_state_dict"])
except Exception as e:
    print(e)

In [ ]:
# Encrypt the model with src=0 (Alice)
dummy_input = torch.empty((1, 3, 224, 224))
private_model = crypten.nn.from_pytorch(dummy_model, dummy_input)
private_model.encrypt(src=0)
print("Model successfully encrypted:", private_model.encrypted)

In [ ]:
BATCH_SIZE = 2

# Create encrypted dataloader with src=1 (Bob)
test_dataset_enc = FlowerDataEnc("test", src=1)
seed = (crypten.mpc.MPCTensor.rand(1) * (2 ** 32)).get_plain_text().int().item()
generator = torch.Generator()
generator.manual_seed(seed)
test_loader_enc = DataLoader(dataset=test_dataset_enc, batch_size=BATCH_SIZE, shuffle=False, collate_fn=crypten_collate, generator=generator)

outs = []
true_ys = []

# Run inference
with torch.no_grad():
    private_model.eval()
    for batch_idx, (xs, ys) in enumerate(test_loader_enc):
        if batch_idx > 0:
            break
        start = time.time()
        if batch_idx == 0:
            for name, param in private_model.named_parameters():
                if 'fc.weight' in name:
                    fc_weights = param.data
                    print(f"\n{name}[:2,:2]:\n{fc_weights[:2,:2]}\n")
            print(f"\nxs[:2,0,0,0]:\n{xs.data[:2,0,0,0]}\n")
            print(f"\nys[:2]:\n{ys[:2].data}\n")
        out = private_model(xs)
        outs.append(out)
        true_ys.append(ys)
        print('(batch %d) time elapsed:' % batch_idx, time.time() - start)
    all_out = crypten.cat(outs, dim=0)
    all_out = all_out.get_plain_text()
    true_ys = crypten.cat(true_ys, dim=0)
    true_ys = true_ys.get_plain_text()
    accuracy = compute_accuracy(all_out, true_ys)
    print("\n\nAccuracy: {0:.4f}".format(accuracy.item()))